## Cargar Librerías

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

## Cargar Datasets

In [2]:
train_data = pd.read_csv('train_dataset.csv')
test_data = pd.read_csv('test_dataset.csv')

## Datos Evaluación

In [3]:
# Separar los datos de evaluación
X_test_user = test_data['user_index']
X_test_book = test_data['book_index']
X_test_features = test_data.drop(['user_index', 'book_index', 'normalized_rating', 'user_id', 'book_id'], axis=1)
y_test = test_data['normalized_rating']

## Datos Entrenamiento

In [4]:
# Separar los datos de entrenamiento
X_train_user = train_data['user_index']
X_train_book = train_data['book_index']
X_train_additional = train_data.drop(['user_index', 'book_index', 'normalized_rating', 'user_id', 'book_id'], axis=1)
y_train = train_data['normalized_rating']

n_users = train_data['user_index'].max() + 1
n_books = train_data['book_index'].max() + 1

## Modelo

In [5]:
def create_model(num_users, num_books):
    # Definir variables clave
    embedding_size = 50  # Tamaño del embedding

    # Input para el filtrado colaborativo (usuarios y libros)
    user_input = keras.layers.Input(shape=(1,), name='user_input')
    book_input = keras.layers.Input(shape=(1,), name='book_input')

    # Embeddings para usuarios y libros
    user_embedding = keras.layers.Embedding(input_dim=num_users, output_dim=embedding_size, name='user_embedding')(user_input)
    book_embedding = keras.layers.Embedding(input_dim=num_books, output_dim=embedding_size, name='book_embedding')(book_input)

    # Aplanar los embeddings
    user_vector = keras.layers.Flatten()(user_embedding)
    book_vector = keras.layers.Flatten()(book_embedding)

    # Concatenación de los embeddings (filtrado colaborativo)
    collaborative_vector = keras.layers.Concatenate()([user_vector, book_vector])

    # Input para características adicionales del libro
    book_features_input = keras.Input(shape=(104,), name='book_features_input')

    # Red densa para procesar las características del contenido del libro
    x = keras.layers.Dense(128, activation='relu')(book_features_input)
    x = keras.layers.Dense(64, activation='relu')(x)

    # Combinar las representaciones colaborativa y de características de contenido
    combined_vector = keras.layers.Concatenate()([collaborative_vector, x])

    # Pasar por capas densas adicionales
    x = keras.layers.Dense(64, activation='relu')(combined_vector)
    x = keras.layers.Dense(32, activation='relu')(x)
    output = keras.layers.Dense(1, activation='sigmoid')(x)

    # Definir el modelo final
    model = keras.Model(inputs=[user_input, book_input, book_features_input], outputs=output)

    # Compilar el modelo
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

    return model

## Crear y Entrenar Modelo

In [6]:
model = create_model(n_users, n_books)
model.fit([X_train_user, X_train_book, X_train_additional], y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
795/795 [==============================] - 4s 4ms/step - loss: 0.0563 - mean_absolute_error: 0.1904 - val_loss: 0.0518 - val_mean_absolute_error: 0.1815
Epoch 2/10
795/795 [==============================] - 3s 3ms/step - loss: 0.0401 - mean_absolute_error: 0.1554 - val_loss: 0.0520 - val_mean_absolute_error: 0.1826
Epoch 3/10
795/795 [==============================] - 3s 4ms/step - loss: 0.0312 - mean_absolute_error: 0.1341 - val_loss: 0.0542 - val_mean_absolute_error: 0.1822
Epoch 4/10
795/795 [==============================] - 3s 3ms/step - loss: 0.0259 - mean_absolute_error: 0.1181 - val_loss: 0.0554 - val_mean_absolute_error: 0.1824
Epoch 5/10
795/795 [==============================] - 3s 3ms/step - loss: 0.0221 - mean_absolute_error: 0.1058 - val_loss: 0.0586 - val_mean_absolute_error: 0.1856
Epoch 6/10
795/795 [==============================] - 3s 3ms/step - loss: 0.0191 - mean_absolute_error: 0.0959 - val_loss: 0.0589 - val_mean_absolute_error: 0.1861
Epoch 7/10
795/7

## Evaluar modelo

In [7]:
y_pred = model.predict([X_test_user, X_test_book, X_test_features])
# Calcular las métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = sqrt(mse)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')

497/497 [==============================] - 1s 974us/step
Mean Squared Error (MSE): 0.06751133912365978
Mean Absolute Error (MAE): 0.19679015340184094


## Optimizar modelo